## Example of a volume-based searchlight

In [ ]:
import AnatSearchlight.searchlight as sl
import numpy as np
import nibabel as nib

To define a searchlight we need:

* a functional mask image (`mask.nii` from the GLM) which specifies the voxels availabe for searchlight analysis.

You can define the searchlight using either:
* a specific radius in mm, or 
* a specific number of voxels. 

To specify only one of these parameters, set the other to `np.inf`. 


In [ ]:
# Give it an appropriate structure name: Used for generating nifti-files for output
mySearchlight = sl.SearchlightSurface('cerebellum')
# This is the mask image for the functional data space
voxel_mask = 'example_data/mask.nii'

# Here we defining
mySearchlight.define(voxel_mask,maxradius=10,maxvoxels=np.inf)
mySearchlight.save('example_data/searchlight_volume.h5')

You can now load the pre-computed searchlight to inspect its properties.

In [ ]:
# Here we wish to inspect the average radius and average number of voxels in each center
mySearchlight = sl.load('searchlight_volume.h5')
print(f'Average size of the searchlight: {mySearchlight.radius.mean():.1f} mm')
print(f'Average number of voxels: {mySearchlight.nvoxels.mean():.1f}')

To run a searchlight we need:

* an MVPA function
* datafiles

The searchlight will calculate the MVPA function on data from datafiles in each center defined in `mySearchlight`.


In [ ]:
# Here we use simple example that calculate the mean activity in the searchlight region.
# The MVPA functions take a `n_images x n_voxels` array as input and return either a scalar or vector as output

def mvpa_mean_function(data):
    # Example of MVPA function that returns a scalar as an output argument
    return np.mean(data)

def mvpa_multi_function(data):
    # Example of MVPA function that returns a vector as an output argument
    return np.mean(data,axis=1)


In [ ]:
## Run the searchlight with the defined scalar MVPA function
datafiles = [f"sub-02_ses-s1_run-01_reg-{s:02d}_beta.nii" for s in range(5)]
results = mySearchlight.run(datafiles, mvpa_mean_function,verbose=False)
mySearchlight.data_to_nifti(results,outfilename='output_scalar.nii')

In [ ]:
## Run the searchlight with the defined scalar vector function
datafiles = [f"sub-02_ses-s1_run-01_reg-{s:02d}_beta.nii" for s in range(5)]
results = mySearchlight.run(datafiles, mvpa_multi_function,verbose=False)
mySearchlight.data_to_nifti(results,outfilename='output_vector.nii')